In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import pandas as pd

# Word embedding - векторное представление слов 

В этом блокноте представлены самые базовые способы представления слов в качестве векторов.

## Bag of words

Рассмотрим самый простой способ приведения текста к набору чисел. Для каждого слова посчитаем, как часто оно встречается в тексте. Результаты запишем в таблицу. Строки будут представлять тексты, столбцы -- слова. Если на пересечении строки с столбца стоит число 5, значит данное слово встретилось в данном тексте 5 раз. В большинстве ячеек будут нули. Поэтому хранить это всё удобнее в виде разреженных матриц (т.е. хранить только ненулевые значения).

Таким образом, при построении "мешка слов" можно выделить следующие действия:

1. Токенизация.

2. Построение словаря: собираем все слова, которые встречались в текстах и пронумеровываем их (по алфавиту, например).

3. Построение разреженной матрицы. В sklearn алгоритм приведения текста в bag-of-words реализован в виде класса CountVectorizer. 


In [3]:
count_vectorizer = CountVectorizer()
texts = ["Великолепный сериал, который поможет успокоить нервы при любых стрессах и просто скрасит серые будни",
         "Пожалуй, если бы я посмотрел только первые пару сезонов этого сериала, я бы с легкой руки написал ему положительную рецензию",
         "В общем, если создатели этого сериала не вернут всё на круги своя, то рейтинги следующих сезонов будут становится все ниже и ниже, а зрительская аудитория будет все меньше и меньше."]

bow = count_vectorizer.fit_transform(texts)
bow.shape

(3, 48)

Результат содержит 3 строки (для 3 текстов) и 48 столбцов (для 48 разных слов). Посмотрим словарь:

In [4]:
count_vectorizer.vocabulary_

{'великолепный': 5,
 'сериал': 36,
 'который': 12,
 'поможет': 27,
 'успокоить': 46,
 'нервы': 20,
 'при': 29,
 'любых': 15,
 'стрессах': 43,
 'просто': 30,
 'скрасит': 39,
 'серые': 38,
 'будни': 2,
 'пожалуй': 25,
 'если': 10,
 'бы': 4,
 'посмотрел': 28,
 'только': 45,
 'первые': 24,
 'пару': 23,
 'сезонов': 35,
 'этого': 47,
 'сериала': 37,
 'легкой': 14,
 'руки': 33,
 'написал': 18,
 'ему': 9,
 'положительную': 26,
 'рецензию': 32,
 'общем': 22,
 'создатели': 41,
 'не': 19,
 'вернут': 6,
 'всё': 8,
 'на': 17,
 'круги': 13,
 'своя': 34,
 'то': 44,
 'рейтинги': 31,
 'следующих': 40,
 'будут': 3,
 'становится': 42,
 'все': 7,
 'ниже': 21,
 'зрительская': 11,
 'аудитория': 0,
 'будет': 1,
 'меньше': 16}

In [5]:
bow.todense()

matrix([[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
         0, 1, 0, 0, 1, 0],
        [0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
         0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 1],
        [1, 1, 0, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 0, 2, 1, 0, 1, 0,
         2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
         1, 0, 1, 0, 0, 1]], dtype=int64)

Как видим, ни стемминга, ни лемматизации по умолчанию не производится.
Поэтому для уменьшения размерности данной матрицы требуется производить лемматизацию/стемминг и удаления стоп-слов. 

**Параметр min_df**

Помимо выше озвученных есть и другие способы отсечения лишнего. Например, можно откидывать слова, которые встречаются слишком редко, с помощью параметра min_df. Установив min_df=2 мы откинем, все слова, которые встречаются менее, чем в 2 документах.

In [6]:
count_vectorizer = CountVectorizer(min_df=2)
bow = count_vectorizer.fit_transform(texts)
count_vectorizer.vocabulary_

{'если': 0, 'сезонов': 1, 'этого': 3, 'сериала': 2}

**Биграммы, триграммы, n-граммы**

По умолчанию bag-of-words (как следует из названия) представляет собой просто мешок слов. То есть для него предложения "It's not good, it's bad!" и "It's not bad, it's good!" абсолютно эквивалентны. Понятно, что при этом теряется много информации. Можно рассматривать не только отдельные слова, а последовательности длиной из 2 слов (биграммы), из 3 слов (триграммы) или в общем случае из n слов (n-граммы). На практике обычно задаётся диапазон от 1 до n.

In [7]:
count_vectorizer = CountVectorizer(ngram_range=(1,2), min_df=2)
bow = count_vectorizer.fit_transform(texts)
print(count_vectorizer.vocabulary_)

print(count_vectorizer.transform(['Если несколько сезонов']).todense())
print(count_vectorizer.transform(['Этого сериала этого сезонов сезонов']).todense())

{'если': 0, 'сезонов': 1, 'этого': 3, 'сериала': 2, 'этого сериала': 4}
[[1 1 0 0 0]]
[[0 2 1 2 1]]


**Ограничение количества признаков**

Понятно, что с ростом n количество выделенных n-грамм быстро растёт. Для ограничения количества признаков можно использовать параметр max_features. В этом случае будет создано не более max_features признаков (будут выбраны самые часто встречающиеся слова и последовательности слов). Например:

In [8]:
count_vectorizer = CountVectorizer(ngram_range=(1,2), max_features=25)
bow = count_vectorizer.fit_transform(texts)
count_vectorizer.vocabulary_

{'поможет': 7,
 'при': 11,
 'просто': 13,
 'поможет успокоить': 8,
 'при любых': 12,
 'просто скрасит': 14,
 'если': 2,
 'бы': 0,
 'посмотрел': 9,
 'сезонов': 21,
 'этого': 23,
 'сериала': 22,
 'руки': 17,
 'положительную': 5,
 'рецензию': 16,
 'посмотрел только': 10,
 'этого сериала': 24,
 'руки написал': 18,
 'положительную рецензию': 6,
 'своя': 19,
 'все': 1,
 'ниже': 4,
 'меньше': 3,
 'своя то': 20,
 'рейтинги следующих': 15}

## TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

$tfidf=tf∗idf$

$idf=\log\frac{(N+1)}{(Nw+1)}+1$

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

In [9]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(texts)
tfidf_vectorizer.vocabulary_

{'великолепный': 5,
 'сериал': 36,
 'который': 12,
 'поможет': 27,
 'успокоить': 46,
 'нервы': 20,
 'при': 29,
 'любых': 15,
 'стрессах': 43,
 'просто': 30,
 'скрасит': 39,
 'серые': 38,
 'будни': 2,
 'пожалуй': 25,
 'если': 10,
 'бы': 4,
 'посмотрел': 28,
 'только': 45,
 'первые': 24,
 'пару': 23,
 'сезонов': 35,
 'этого': 47,
 'сериала': 37,
 'легкой': 14,
 'руки': 33,
 'написал': 18,
 'ему': 9,
 'положительную': 26,
 'рецензию': 32,
 'общем': 22,
 'создатели': 41,
 'не': 19,
 'вернут': 6,
 'всё': 8,
 'на': 17,
 'круги': 13,
 'своя': 34,
 'то': 44,
 'рейтинги': 31,
 'следующих': 40,
 'будут': 3,
 'становится': 42,
 'все': 7,
 'ниже': 21,
 'зрительская': 11,
 'аудитория': 0,
 'будет': 1,
 'меньше': 16}

Словарь содержит те же 48 значений, которые были бы и для CountVectorizer. Но значения в таблице другие:

In [10]:
tfidf.todense()

matrix([[0.        , 0.        , 0.2773501 , 0.        , 0.        ,
         0.2773501 , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.2773501 , 0.        , 0.        ,
         0.2773501 , 0.        , 0.        , 0.        , 0.        ,
         0.2773501 , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.2773501 , 0.        , 0.2773501 ,
         0.2773501 , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.2773501 , 0.        , 0.2773501 , 0.2773501 ,
         0.        , 0.        , 0.        , 0.2773501 , 0.        ,
         0.        , 0.2773501 , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.48065817,
         0.        , 0.        , 0.        , 0.        , 0.24032909,
         0.18277647, 0.        , 0.        , 0.        , 0.24032909,
         0.        , 0.        , 0.        , 0.24032909, 0.        ,
         0.        , 0.        , 0.        , 0.24032909, 

Ненулевые значения находятся на тех же местах, но отмасштабированы в зависимости от частоты слов.

**Параметр sublinear_tf**

Большая часть параметров у CountVectorizer и TfidfVectorizer одинакова. Но у TfidfVectorizer есть один важный дополнительный параметр.

Как видно из формулы tfidf = tf * idf, если слово будет встречаться не один, а два раза, то tfidf вырастет в два раза. Если слово будет встречаться не один, а 10 раз, то tfidf вырастет почти в 10 раз. В качестве примера добавим в третью строку ещё пару слов меньше

In [11]:
texts = ["Великолепный сериал, который поможет успокоить нервы при любых стрессах и просто скрасит серые будни",
         "Пожалуй, если бы я посмотрел только первые пару сезонов этого сериала, я бы с легкой руки написал ему положительную рецензию",
         "В общем, если создатели этого сериала не вернут всё на круги своя, то рейтинги следующих сезонов будут становится все ниже и ниже, а зрительская аудитория будет все меньше и меньше и меньше и меньше."]
TfidfVectorizer().fit_transform(texts).todense()[2]         

matrix([[0.15373049, 0.15373049, 0.        , 0.15373049, 0.        ,
         0.        , 0.15373049, 0.30746099, 0.15373049, 0.        ,
         0.116916  , 0.15373049, 0.        , 0.15373049, 0.        ,
         0.        , 0.61492198, 0.15373049, 0.        , 0.15373049,
         0.        , 0.30746099, 0.15373049, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.15373049, 0.        , 0.        , 0.15373049,
         0.116916  , 0.        , 0.116916  , 0.        , 0.        ,
         0.15373049, 0.15373049, 0.15373049, 0.        , 0.15373049,
         0.        , 0.        , 0.116916  ]])

Значение tfidf слова "меньше" выросло с 0.36325471 до 0.61492198, а остальные упали .

Вопрос - хотим ли мы таких сильных изменений. Если не хотим, то можно использовать параметр sublinear_tf=True. При его использовании вместо tf будет браться 1 + log(tf). То есть по-прежнему с ростом tf будет расти и tfidf, но уже не так радикально (и соответственно остальные значения будут уменьшаться не так быстро). Для некоторых задач это может дать прирост в качестве.

In [12]:
TfidfVectorizer(sublinear_tf=True).fit_transform(texts).todense()[2]

matrix([[0.18336592, 0.18336592, 0.        , 0.18336592, 0.        ,
         0.        , 0.18336592, 0.31046549, 0.18336592, 0.        ,
         0.13945451, 0.18336592, 0.        , 0.18336592, 0.        ,
         0.        , 0.43756505, 0.18336592, 0.        , 0.18336592,
         0.        , 0.31046549, 0.18336592, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.18336592, 0.        , 0.        , 0.18336592,
         0.13945451, 0.        , 0.13945451, 0.        , 0.        ,
         0.18336592, 0.18336592, 0.18336592, 0.        , 0.18336592,
         0.        , 0.        , 0.13945451]])

## Задание

1. Создать CountVectorizer и TfidfVectorizer для ваших корпусов документов.
2. Сериализовать и сохранить на диск эти модели.
3. Дописать в csv два дополнительных столбца с результатами преобразования с помощью двух методов.


# Предобработка данных

## Загрузка данных

In [13]:
name = '../data/vacancy.csv'
df = pd.read_csv(name, sep=",")
df

,vacancy_id,json,description,tokens,CountVectorizer,TfIdf
0,33551858.0,"{'id': '33551858', 'premium': False, 'billing_...",<p> </p> <p>Приглашаем на серьёзную ответствен...,приглаша серьезн ответствен работ приветств ли...,"(0, 12724)\t1\n (0, 15957)\t1\n (0, 10475)...","(0, 23147)\t0.06890922586156473\n (0, 25893..."
1,33551859.0,"{'id': '33551859', 'premium': False, 'billing_...",<p><strong><em>Департамент управления барами –...,департамент управлен бар – « вкусн » подраздел...,"(0, 12724)\t1\n (0, 10475)\t1\n (0, 13676)...","(0, 11687)\t0.06752638280049036\n (0, 18277..."
2,33551860.0,"{'id': '33551860', 'premium': False, 'billing_...",<p>«Сеть фитнес клубов ФизКульт является призн...,« сет фитнес клуб физкульт явля призна лидер ф...,"(0, 13676)\t3\n (0, 12697)\t1\n (0, 7552)\...","(0, 21529)\t0.05084381221092673\n (0, 19002..."
3,33551861.0,"{'id': '33551861', 'premium': False, 'billing_...",<p><strong>Должностные обязанности:</strong></...,должностн обязан • осуществлен расчетно-кассов...,"(0, 13676)\t3\n (0, 9656)\t1\n (0, 18038)\...","(0, 20618)\t0.07438251902005348\n (0, 12207..."
4,33551862.0,"{'id': '33551862', 'premium': False, 'billing_...",<p><strong>Обязанности:</strong></p> <ul> <li>...,обязан амбулаторно-поликлиническ требован нали...,"(0, 9656)\t1\n (0, 18038)\t1\n (0, 8544)\t...","(0, 23196)\t0.2476664072242917\n (0, 9135)\..."
...,...,...,...,...,...,...
9995,33587501.0,"{'id': '33587505', 'premium': False, 'billing_...",<p><em><strong>ИТАЛЬЯНСКАЯ ОБУВНАЯ ФАБРИКА МИЗ...,итальянск обувн фабрик мизр групп ищет рабоч о...,"(0, 10475)\t1\n (0, 13676)\t2\n (0, 12164)...","(0, 27132)\t0.17458989575459807\n (0, 19533..."
9996,33587502.0,"{'id': '33587506', 'premium': False, 'billing_...",<p>В ООО <strong>«Канц-Лайн»</strong> - компан...,оо « канц-лайн » компан снабжа предприят крым ...,"(0, 10475)\t1\n (0, 13676)\t8\n (0, 9656)\...","(0, 11596)\t0.10755976021458305\n (0, 9720)..."
9997,33587503.0,"{'id': '33587507', 'premium': False, 'billing_...",<p>В ООО <strong>«Канц-Лайн»</strong> - компан...,оо « канц-лайн » компан снабжа предприят крым ...,"(0, 10475)\t1\n (0, 13676)\t8\n (0, 9656)\...","(0, 11596)\t0.10763021245405788\n (0, 9720)..."
9998,33587504.0,"{'id': '33587508', 'premium': False, 'billing_...",<p>Сеть магазинов товаров для животных Мир Хво...,сет магазин товар животн мир хвостат интернет-...,"(0, 13676)\t3\n (0, 7552)\t1\n (0, 9656)\t...","(0, 26494)\t0.27491273354927076\n (0, 203)\..."


## Bag of words

Создадим два столбца - для каждого из словарей

In [14]:
count_vectorizer = CountVectorizer()
bow = count_vectorizer.fit_transform(df.tokens)

In [15]:
bow.shape

(10000, 27902)

In [16]:
count_vectorizer.vocabulary_

{'приглаша': 19913,
 'серьезн': 22652,
 'ответствен': 17408,
 'работ': 20874,
 'приветств': 19870,
 'личн': 14012,
 'качеств': 12273,
 'порядочн': 19332,
 'исполнительн': 11836,
 'быстр': 7151,
 'обучаем': 16698,
 'обязан': 16790,
 'треб': 25071,
 'лицензирова': 14009,
 'охранник': 17737,
 'объек': 16758,
 'находя': 15776,
 'ул': 25590,
 'калинин': 12034,
 'красноярск': 13376,
 'охра': 17734,
 'объект': 16759,
 'имуществ': 11511,
 'собственник': 23165,
 'город': 9008,
 'разн': 21109,
 'район': 21178,
 'грб': 9162,
 'водител': 7849,
 'гбр': 8685,
 'стаж': 23697,
 'вожден': 7904,
 'мен': 14667,
 '3х': 657,
 'лет': 13879,
 'старш': 23767,
 'смен': 23035,
 'разряд': 21158,
 'обязательн': 16802,
 'навык': 15456,
 'способн': 23585,
 'реша': 21747,
 'сложн': 22986,
 'ситуац': 22800,
 'готовн': 9097,
 'люб': 14163,
 'момент': 15141,
 'выеха': 8341,
 'сигнал': 22707,
 'quot': 4029,
 'тревог': 25091,
 'вахтов': 7299,
 'метод': 14796,
 'нескольк': 16103,
 'направлен': 15643,
 'требован': 25073,
 

Как мы видим, после применения стемминга алгоритм работает достаточно хорошо

In [17]:
bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [2, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [2, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Однако, слов все еще слишком много. На 10000 документов можно откинуть слова, которые встречаются достаточно редко. Также обратим внимание на n-граммы. Учитывая, что предложения по работе достаточно разные, достаточно будет рассматривать одиночные слова, биграммы, триграммы.

In [18]:
count_vectorizer = CountVectorizer(ngram_range=(1,3), min_df=10, max_features=20000)
bow = count_vectorizer.fit_transform(df.tokens)

In [19]:
bow.shape

(10000, 20000)

In [20]:
count_vectorizer.vocabulary_

{'приглаша': 12724,
 'серьезн': 15957,
 'ответствен': 10475,
 'работ': 13676,
 'приветств': 12697,
 'личн': 7552,
 'качеств': 6035,
 'порядочн': 12164,
 'исполнительн': 5727,
 'быстр': 1861,
 'обучаем': 9484,
 'обязан': 9656,
 'треб': 18007,
 'лицензирова': 7549,
 'охранник': 10935,
 'находя': 8720,
 'ул': 18513,
 'красноярск': 7176,
 'охра': 10928,
 'объект': 9611,
 'имуществ': 5496,
 'собственник': 16347,
 'город': 3292,
 'разн': 14655,
 'район': 14750,
 'водител': 2404,
 'стаж': 17005,
 'вожден': 2440,
 'мен': 7943,
 '3х': 453,
 'лет': 7381,
 'старш': 17081,
 'смен': 16215,
 'разряд': 14739,
 'обязательн': 9813,
 'навык': 8440,
 'способн': 16827,
 'реша': 15147,
 'сложн': 16168,
 'ситуац': 16062,
 'готовн': 3386,
 'люб': 7636,
 'момент': 8334,
 'quot': 805,
 'вахтов': 1959,
 'метод': 8136,
 'нескольк': 8962,
 'направлен': 8633,
 'требован': 18038,
 'налич': 8544,
 'диплом': 3876,
 'удостоверен': 18475,
 'услов': 18721,
 'график': 3437,
 'уровен': 18664,
 'заработн': 5020,
 'плат': 1

Добавим столбец `CountVectorizer` как результат работы мешка слов

In [21]:
df = df.assign(**{'CountVectorizer': list(bow)})

Посмотрим на получившийся датафрейм

In [22]:
df

,vacancy_id,json,description,tokens,CountVectorizer,TfIdf
0,33551858.0,"{'id': '33551858', 'premium': False, 'billing_...",<p> </p> <p>Приглашаем на серьёзную ответствен...,приглаша серьезн ответствен работ приветств ли...,"(0, 12724)\t1\n (0, 15957)\t1\n (0, 10475)...","(0, 23147)\t0.06890922586156473\n (0, 25893..."
1,33551859.0,"{'id': '33551859', 'premium': False, 'billing_...",<p><strong><em>Департамент управления барами –...,департамент управлен бар – « вкусн » подраздел...,"(0, 12724)\t1\n (0, 10475)\t1\n (0, 13676)...","(0, 11687)\t0.06752638280049036\n (0, 18277..."
2,33551860.0,"{'id': '33551860', 'premium': False, 'billing_...",<p>«Сеть фитнес клубов ФизКульт является призн...,« сет фитнес клуб физкульт явля призна лидер ф...,"(0, 13676)\t3\n (0, 12697)\t1\n (0, 7552)\...","(0, 21529)\t0.05084381221092673\n (0, 19002..."
3,33551861.0,"{'id': '33551861', 'premium': False, 'billing_...",<p><strong>Должностные обязанности:</strong></...,должностн обязан • осуществлен расчетно-кассов...,"(0, 13676)\t3\n (0, 9656)\t1\n (0, 18038)\...","(0, 20618)\t0.07438251902005348\n (0, 12207..."
4,33551862.0,"{'id': '33551862', 'premium': False, 'billing_...",<p><strong>Обязанности:</strong></p> <ul> <li>...,обязан амбулаторно-поликлиническ требован нали...,"(0, 9656)\t1\n (0, 18038)\t1\n (0, 8544)\t...","(0, 23196)\t0.2476664072242917\n (0, 9135)\..."
...,...,...,...,...,...,...
9995,33587501.0,"{'id': '33587505', 'premium': False, 'billing_...",<p><em><strong>ИТАЛЬЯНСКАЯ ОБУВНАЯ ФАБРИКА МИЗ...,итальянск обувн фабрик мизр групп ищет рабоч о...,"(0, 10475)\t1\n (0, 13676)\t2\n (0, 12164)...","(0, 27132)\t0.17458989575459807\n (0, 19533..."
9996,33587502.0,"{'id': '33587506', 'premium': False, 'billing_...",<p>В ООО <strong>«Канц-Лайн»</strong> - компан...,оо « канц-лайн » компан снабжа предприят крым ...,"(0, 10475)\t1\n (0, 13676)\t8\n (0, 9656)\...","(0, 11596)\t0.10755976021458305\n (0, 9720)..."
9997,33587503.0,"{'id': '33587507', 'premium': False, 'billing_...",<p>В ООО <strong>«Канц-Лайн»</strong> - компан...,оо « канц-лайн » компан снабжа предприят крым ...,"(0, 10475)\t1\n (0, 13676)\t8\n (0, 9656)\...","(0, 11596)\t0.10763021245405788\n (0, 9720)..."
9998,33587504.0,"{'id': '33587508', 'premium': False, 'billing_...",<p>Сеть магазинов товаров для животных Мир Хво...,сет магазин товар животн мир хвостат интернет-...,"(0, 13676)\t3\n (0, 7552)\t1\n (0, 9656)\t...","(0, 26494)\t0.27491273354927076\n (0, 203)\..."


## TF-IDF

Параметр `sublinear_tf` отвечает за изменение оценки `tf`: вместо $tf$ берется $1 + log(tf)$

In [23]:
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True)
tfidf = tfidf_vectorizer.fit_transform(df.tokens)
tfidf.shape

(10000, 27902)

In [24]:
tfidf.todense()[0]

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [25]:
df = df.assign(**{'TfIdf': list(tfidf)})

Посмотрим на получившийся датафрейм

In [26]:
df

,vacancy_id,json,description,tokens,CountVectorizer,TfIdf
0,33551858.0,"{'id': '33551858', 'premium': False, 'billing_...",<p> </p> <p>Приглашаем на серьёзную ответствен...,приглаша серьезн ответствен работ приветств ли...,"(0, 12724)\t1\n (0, 15957)\t1\n (0, 10475)...","(0, 23147)\t0.06890922586156473\n (0, 25893..."
1,33551859.0,"{'id': '33551859', 'premium': False, 'billing_...",<p><strong><em>Департамент управления барами –...,департамент управлен бар – « вкусн » подраздел...,"(0, 12724)\t1\n (0, 10475)\t1\n (0, 13676)...","(0, 11687)\t0.06752638280049036\n (0, 18277..."
2,33551860.0,"{'id': '33551860', 'premium': False, 'billing_...",<p>«Сеть фитнес клубов ФизКульт является призн...,« сет фитнес клуб физкульт явля призна лидер ф...,"(0, 13676)\t3\n (0, 12697)\t1\n (0, 7552)\...","(0, 21529)\t0.05084381221092673\n (0, 19002..."
3,33551861.0,"{'id': '33551861', 'premium': False, 'billing_...",<p><strong>Должностные обязанности:</strong></...,должностн обязан • осуществлен расчетно-кассов...,"(0, 13676)\t3\n (0, 9656)\t1\n (0, 18038)\...","(0, 20618)\t0.07438251902005348\n (0, 12207..."
4,33551862.0,"{'id': '33551862', 'premium': False, 'billing_...",<p><strong>Обязанности:</strong></p> <ul> <li>...,обязан амбулаторно-поликлиническ требован нали...,"(0, 9656)\t1\n (0, 18038)\t1\n (0, 8544)\t...","(0, 23196)\t0.2476664072242917\n (0, 9135)\..."
...,...,...,...,...,...,...
9995,33587501.0,"{'id': '33587505', 'premium': False, 'billing_...",<p><em><strong>ИТАЛЬЯНСКАЯ ОБУВНАЯ ФАБРИКА МИЗ...,итальянск обувн фабрик мизр групп ищет рабоч о...,"(0, 10475)\t1\n (0, 13676)\t2\n (0, 12164)...","(0, 27132)\t0.17458989575459807\n (0, 19533..."
9996,33587502.0,"{'id': '33587506', 'premium': False, 'billing_...",<p>В ООО <strong>«Канц-Лайн»</strong> - компан...,оо « канц-лайн » компан снабжа предприят крым ...,"(0, 10475)\t1\n (0, 13676)\t8\n (0, 9656)\...","(0, 11596)\t0.10755976021458305\n (0, 9720)..."
9997,33587503.0,"{'id': '33587507', 'premium': False, 'billing_...",<p>В ООО <strong>«Канц-Лайн»</strong> - компан...,оо « канц-лайн » компан снабжа предприят крым ...,"(0, 10475)\t1\n (0, 13676)\t8\n (0, 9656)\...","(0, 11596)\t0.10763021245405788\n (0, 9720)..."
9998,33587504.0,"{'id': '33587508', 'premium': False, 'billing_...",<p>Сеть магазинов товаров для животных Мир Хво...,сет магазин товар животн мир хвостат интернет-...,"(0, 13676)\t3\n (0, 7552)\t1\n (0, 9656)\t...","(0, 26494)\t0.27491273354927076\n (0, 203)\..."


## Сохранение датафрейма

In [27]:
df.to_csv(name, sep=',', index=False)

# Вывод

Были изучены методики векторного представления слов. Наиболее популярные - мешок слов и tf-idf